# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v2.1) is available! Your current version is v1.10.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to 821691892@qq.com and will expire on July 05, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: C:\Users\lixx3527\AppData\Local\Temp\graphlab_server_1471443278.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [8]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.462599     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.498610     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.get('coefficients').print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [5]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [52]:
import numpy as np
l1_penalty_array=np.logspace(1,7,num=13)

rss=0
model_1=graphlab.linear_regression.create(training,target='price',features=all_features,
                                          validation_set=None,l2_penalty=0., l1_penalty=l1_penalty_array[0],verbose=False)
pres=model_1.predict(validation)
for i in range(len(pres)):
    rss+=(pres[i]-validation['price'][i])**2
min_rss=rss
min_index=l1_penalty_array[0]
print min_rss

6.25766285142e+14


In [53]:
rss_array=[]
min_index=0
j=1
for ele in l1_penalty_array:
    #j=0
    rss=0
    model_1=graphlab.linear_regression.create(training,target='price',features=all_features,
                                          validation_set=None,l2_penalty=0., l1_penalty=ele)
    pres=model_1.predict(validation)
    rmse=graphlab.evaluation.rmse(validation['price'],pres)
    rss=rmse*rmse*len(pres)
    rss_array.append(rss)
    
#print min_rss,min_index
print rss_array

        
    

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.268020     | 5659149.988446     | 285408.885485 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

[625766285142461.0, 625766285362394.0, 625766286057884.5, 625766288257226.8, 625766295212186.1, 625766317206078.5, 625766386760657.0, 625766606749278.8, 625767302791633.5, 625769507643886.9, 625776517727024.5, 625799062845468.8, 625883719085424.0]


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [67]:
index=0
min=rss_array[0]
j=0
for e in rss_array[0:len(rss_array)]:
    if min>=e:
        min=e
        index=j
        print "can we get here"
    j+=1
print index,min

#rss=0
model=graphlab.linear_regression.create(training,target='price',features=all_features,
                                          validation_set=None,l2_penalty=0., l1_penalty=l1_penalty_array[0],verbose=False)

model['coefficients']['value'].nnz()

can we get here
0 6.25766285142e+14


18

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [68]:
model.get('coefficients').print_rows(num_rows=18)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [25]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [70]:
import numpy as np
l1_penalty_values = np.logspace(8, 10, num=20)
model_all['coefficients']['value'][1:len(all_features)].nnz()

5

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [71]:

num_array=[]
for ele in l1_penalty_values:
    model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=ele,
                                           validation_set=None)
    num=model['coefficients']['value'].nnz()
    num_array.append(num)
print num_array
#print num_array

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000012  | 0.255176     | 7187431.218291     | 380705.223778 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


This model may not be optimal. To improve it, consider increasing `max_iterations`.

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [72]:
l1_penalty_min =0 
l1_penalty_max =0

if num_array[0]<=max_nonzeros:
    l1_penalty_min=l1_penalty_values[0]
    #print "can we get here"


for i in range(len(num_array)):
    if num_array[i]>max_nonzeros and i+1<len(num_array) and num_array[i+1]<=7:
        l1_penalty_min=l1_penalty_values[i]
        break
    elif i+1==len(num_array):
        l1_penalty_min=l1_penalty_values[i]
        break

#print l1_penalty_min

x=range(len(num_array))
x.reverse()

if num_array[x[0]]>=max_nonzeros:
    l1_penalty_max=l1_penalty_values[x[0]]


for i in x:
    if num_array[i]<max_nonzeros and i-1>=0 and num_array[i-1]>=max_nonzeros:
        l1_penalty_max=l1_penalty_values[i]
        break
    elif i-1<0:
        l1_penalty_max=l1_penalty_values[0]
        break

if l1_penalty_max>l1_penalty_min:
    print l1_penalty_min,l1_penalty_max

print l1_penalty_values[14],l1_penalty_values[15]
    

2976351441.63 3792690190.73
2976351441.63 3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [73]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [74]:
import sys
element=[]
element_index=[]
ele_min=0
for ele in l1_penalty_values:
    model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=ele,
                                           validation_set=None,verbose = False)
    if model['coefficients']['value'].nnz()==7:
        #print "can we get here"
        pres=model.predict(validation)
        rmse=graphlab.evaluation.rmse(validation['price'],pres)
        rss=rmse*rmse*len(pres)
        element_index.append(ele)
        element.append(rss)
        

print element
    
print element_index


print np.array(element).min()
    
    

[1046937488751714.4, 1051147625612859.8, 1055992735343002.2, 1060799531763288.2]
[3448968612.1634364, 3491933809.4845386, 3534899006.8056407, 3577864204.1267428]
1.04693748875e+15


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [76]:
model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=element_index[0],
                                           validation_set=None,verbose = False)



In [77]:
model.get('coefficients').print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |

In [21]:
x.reverse()

In [22]:
x

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [37]:
2*3


6

In [54]:
print rss_array

[625766285142461.0, 625766285362394.0, 625766286057884.5, 625766288257226.8, 625766295212186.1, 625766317206078.5, 625766386760657.0, 625766606749278.8, 625767302791633.5, 625769507643886.9, 625776517727024.5, 625799062845468.8, 625883719085424.0]


In [56]:
x=np.array(rss_array).min()

In [58]:
rss_array.index(x)

0